In [1]:
!git clone https://github.com/AvonYangXX1/AMPLify-Feedback.git
import tensorflow as tf
import numpy as np
import pandas as pd

Cloning into 'AMPLify-Feedback'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 106 (delta 66), reused 98 (delta 61), pack-reused 0
Receiving objects: 100% (106/106), 6.40 MiB | 2.97 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (41/41), done.


In [2]:
tf.config.run_functions_eagerly(True)

seq_train = np.load("AMPLify-Feedback/processed_data/cv/seq_train_1.npy")
state_train = np.load("AMPLify-Feedback/processed_data/cv/state_train_1.npy")
label_train = np.load("AMPLify-Feedback/processed_data/cv/label_train_1.npy")
seq_test = np.load("AMPLify-Feedback/processed_data/cv/seq_test_1.npy")
state_test = np.load("AMPLify-Feedback/processed_data/cv/state_test_1.npy")
label_test = np.load("AMPLify-Feedback/processed_data/cv/label_test_1.npy")
seq_train = np.expand_dims(seq_train, 2).astype("float32")
seq_test = np.expand_dims(seq_test, 2).astype("float32")
state_train = np.expand_dims(state_train, 2).astype("float32")
state_test = np.expand_dims(state_test, 2).astype("float32")

train_dataset = tf.data.Dataset.from_tensor_slices(((seq_train, state_train), label_train))
train_dataset = train_dataset.shuffle(buffer_size=len(seq_train))
train_dataset = train_dataset.batch(100)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices(((seq_test, state_test), label_test))
test_dataset = test_dataset.batch(100)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
@tf.function
def train_step(seqs, states, labels):
    with tf.GradientTape() as tape:
        predictions = model(seqs, states)
        loss = loss_function(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuracy_metric.update_state(labels, predictions)
    return loss

@tf.function
def val_step(seqs, states, labels):
    predictions = model(seqs, states)
    loss = loss_function(labels, predictions)
    val_accuracy_metric.update_state(labels, predictions)
    return loss

In [4]:
class ActivityPredictor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.Embed = tf.keras.models.Sequential([
            # tf.keras.layers.Conv1D(1, 3, activation='relu', name="Conv1D_0"),
            # tf.keras.layers.Conv1D(1, 1, activation='relu', name="Conv1D_Fusion"),
            tf.keras.layers.Flatten(name="Flatten"),
            tf.keras.layers.Dense(512, activation="relu", name="EmbedState"),
            # tf.keras.layers.LayerNormalization(name="LayerNorm_0"),
        ], name="EmbedState")

        self.cnn = tf.keras.models.Sequential([
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_0"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_1"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_2"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_3"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_4"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_5"),
            tf.keras.layers.Conv1D(64, 3, activation='relu', name="Conv1D_6"),
            tf.keras.layers.Conv1D(1, 1, activation='relu', name="Conv1D_Fusion"),
            tf.keras.layers.Flatten(name="Flatten"),
            # tf.keras.layers.LSTM(1024, name="GRU_0", return_sequences=True),
            # tf.keras.layers.GRU(1024, name="GRU_1", return_sequences=True),
            # tf.keras.layers.GRU(128, name="GRU_2", return_sequences=True),
            # tf.keras.layers.GRU(128, name="GRU_3", return_sequences=True),
            # tf.keras.layers.LSTM(1024, name="GRU_4", return_sequences=False),
            tf.keras.layers.Dense(512, activation="relu", name="CNN_Out"),
            # tf.keras.layers.LayerNormalization(name="LayerNorm_1"),
        ], name="cnn")

        self.fcn = tf.keras.models.Sequential([
            tf.keras.layers.Dense(1024, activation="relu", name="Dense_0"),
            tf.keras.layers.Dense(1024, activation="relu", name="Dense_1"),
            tf.keras.layers.Dense(1024, activation="relu", name="Dense_2"),
            tf.keras.layers.Dense(1024, activation="relu", name="Dense_3"),
            tf.keras.layers.Dense(64, activation="relu", name="Dense_4"),
            # tf.keras.layers.LayerNormalization(name="LayerNorm_3"),
        ], name="fcn")
        self.Output = tf.keras.layers.Dense(1, activation="sigmoid", name="Output")
        # self.call_layernorm = tf.keras.layers.LayerNormalization(name="LayerNorm_2")

    def call(self, seq, state):
        x0 = self.cnn(seq)
        x1 = self.Embed(state)
        x = tf.keras.layers.Concatenate()([x0, x1])
        # x = self.call_layernorm(x)
        x = self.fcn(x)
        x = self.Output(x)
        return x

model = ActivityPredictor()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_function = tf.keras.losses.BinaryCrossentropy()
train_accuracy_metric = tf.keras.metrics.BinaryAccuracy()
val_accuracy_metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss_function, metrics=["accuracy"])

In [5]:
EPOCHS = 20

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print('-' * 40)

    # Training Loop
    total_loss = 0
    num_batches = 0
    for (batch_seqs, batch_states), batch_labels in train_dataset:
        total_loss += train_step(batch_seqs, batch_states, batch_labels)
        num_batches += 1
    train_loss = total_loss / num_batches
    train_accuracy = train_accuracy_metric.result()

    # Validation Loop
    total_val_loss = 0
    num_val_batches = 0
    for (batch_seqs, batch_states), batch_labels in test_dataset:
        total_val_loss += val_step(batch_seqs, batch_states, batch_labels)
        num_val_batches += 1
    val_loss = total_val_loss / num_val_batches
    val_accuracy = val_accuracy_metric.result()

    # Logging Results
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Resetting Metrics
    train_accuracy_metric.reset_states()
    val_accuracy_metric.reset_states()


Epoch 1/20
----------------------------------------
Train Loss: 0.4965, Train Accuracy: 0.4093
Validation Loss: 0.5422, Validation Accuracy: 0.3090

Epoch 2/20
----------------------------------------
Train Loss: 0.3874, Train Accuracy: 0.4344
Validation Loss: 0.3637, Validation Accuracy: 0.4444

Epoch 3/20
----------------------------------------
Train Loss: 0.3278, Train Accuracy: 0.4479
Validation Loss: 0.3161, Validation Accuracy: 0.4394

Epoch 4/20
----------------------------------------
Train Loss: 0.2876, Train Accuracy: 0.4609
Validation Loss: 0.2906, Validation Accuracy: 0.4437

Epoch 5/20
----------------------------------------
Train Loss: 0.2507, Train Accuracy: 0.4688
Validation Loss: 0.3160, Validation Accuracy: 0.4229

Epoch 6/20
----------------------------------------
Train Loss: 0.2251, Train Accuracy: 0.4750
Validation Loss: 0.2433, Validation Accuracy: 0.4664

Epoch 7/20
----------------------------------------
Train Loss: 0.2053, Train Accuracy: 0.4785
Validation

In [16]:
model(batch_seqs, batch_states)

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.48231667],
       [0.48240632],
       [0.48240334],
       [0.48240334],
       [0.48238984],
       [0.48054528],
       [0.48234296],
       [0.48237464],
       [0.4823702 ],
       [0.4823702 ],
       [0.48231667],
       [0.48222545],
       [0.4823702 ],
       [0.48234275],
       [0.4823702 ],
       [0.48231667],
       [0.48231667],
       [0.48240632],
       [0.4823702 ],
       [0.48207644],
       [0.48234275],
       [0.48240334],
       [0.48236373],
       [0.48232865],
       [0.4822315 ],
       [0.48215923],
       [0.4823702 ],
       [0.48234275],
       [0.48240632],
       [0.48231667],
       [0.48240632],
       [0.48231667]], dtype=float32)>

In [70]:
print(batch_labels)

tf.Tensor(
[[0.99735728]
 [0.99637688]
 [0.9969381 ]
 [0.99495547]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.99190468]
 [0.99623838]
 [0.99704865]
 [0.        ]
 [0.99362883]
 [0.99620954]
 [0.        ]
 [0.51013666]
 [0.98885122]
 [0.99579429]
 [0.97980182]
 [0.        ]
 [0.76107271]
 [0.99717378]
 [0.        ]
 [0.99703239]
 [0.99319677]
 [0.99507637]
 [0.99303711]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.99195264]
 [0.99514576]
 [0.98905406]
 [0.        ]
 [0.99711159]
 [0.9872774 ]
 [0.67043978]
 [0.        ]
 [0.99143798]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.99571731]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.99708682]
 [0.        ]
 [0.        ]
 [0.99514576]
 [0.        ]
 [0.9910069 ]
 [0.99643615]
 [0.        ]
 [0.        ]
 [0.9969701 ]
 [0.99721562]
 [0.        ]
 [0.86215229]
 [0.99661139]
 [0.        ]
 [0.        ]
 [0.99613608]
 [0.        ]
 [0.        ]
 [0.99725685]
 [0.99362